Create Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Read the data

In [2]:
import numpy as np
import pandas as pd
movie = pd.read_csv('drive/My Drive/FTMLE - Tonga/Data/movie_review.csv',encoding='utf-8', sep='\t')
movie.head(20)

,id,review,sentiment
0,5814_8,With all this stuff going down at the moment w...,1
1,2381_9,"\The Classic War of the Worlds\"" by Timothy Hi...",1
2,7759_3,The film starts with a manager (Nicholas Bell)...,0
3,3630_4,It must be assumed that those who praised this...,0
4,9495_8,Superbly trashy and wondrously unpretentious 8...,1
5,8196_8,I dont know why people think this is such a ba...,1
6,7166_2,"This movie could have been very good, but come...",0
7,319_1,"A friend of mine bought this film for Â£1, and...",0
8,8713_10,<br /><br />This movie is full of references. ...,1
9,2486_3,"What happens when an army of wetbacks, towelhe...",0


Import "stopwords" from nltk library

In [3]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

Clean & Vectorize Data

In [0]:
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
porter = PorterStemmer()
stop = stopwords.words('english')

def tokenizer_porter(text):
    
    return [porter.stem(word) for word in text.split()]

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    return text

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

In [0]:
movie['review'] = movie['review'].apply(preprocessor)

Check data again

In [8]:
movie.sample(10)

,id,review,sentiment
18766,5568_8,a typical romp through cheech and chong s real...,1
9076,12131_7,the movie concerns about philip leslie howard ...,1
13860,11641_7,i just caught an episode about brad the crack ...,1
4086,2061_1,the only film i ve ever walked out on amazing ...,0
1019,2741_2,spoiler i love branagh love helena bonham cart...,0
10228,11261_4,to start with i have to point out the fact tha...,0
3768,1149_3,the fbi story 1959 was warner bros 149 minute ...,0
5687,192_2,the only good part about this film is the beau...,0
3023,12099_4,i ve been a fan of all things bill maher for 1...,0
1931,10559_8,i haven t written a review on imdb for the lon...,1


Train the model

In [0]:
from sklearn.model_selection import train_test_split

X = movie['review']
y = movie['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7feb9ce4f840>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7feb9ce4f620>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

Test for model accuracy

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.892

Print out the final report 

In [15]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.90      0.88      0.89      2230
           1       0.88      0.91      0.89      2270

    accuracy                           0.89      4500
   macro avg       0.89      0.89      0.89      4500
weighted avg       0.89      0.89      0.89      4500



In [0]:
mveva = pd.read_csv('drive/My Drive/FTMLE - Tonga/Data/movie_review_evaluation.csv',encoding='utf-8', sep='\t')



In [0]:
mvtest = mveva['review']

In [0]:
predictions1 = clf.predict(mvtest)

Save the test Data

In [0]:
prediclist = predictions1.tolist()
mveva['sentiment'] = prediclist
mveva.to_csv('kha.csv', index = False)